In [1]:
from requests import get, post
from bs4 import BeautifulSoup
import os
import zipfile
import numpy as np
import pandas as pd

In [156]:
cate_list = ['2_dog_fence', '3_dog_nosework', '4_cat_tower', '5_cat_scratcher', 
             '6_cat_toilet' , '7_house', '8_carrier', '9_clothes']

cate_url_dict = {'2_dog_fence':['https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975312011%2Cn%3A2975346011&dc&ds=v1%3AO%2FdUmSDs6UXM8q5ZBiltLCyOoeiZgkTDYisLdagbcJA&qid=1702140570&rnid=2619533011&ref=sr_nr_n_10'], 
                 '3_dog_nosework':['https://www.amazon.com/s?k=nosework+for+dogs&crid=RN4YBTCU4RYK&qid=1702208454&sprefix=nosework%2Caps%2C253&ref=sr_pg_1'], 
                 '4_cat_tower':['https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975241011%2Cn%3A2975243011%2Cn%3A2975248011&dc&ds=v1%3AiccSBzb%2B7zvuTcI2RKh0K%2FbyVTJ76wnghF2MltPS840&qid=1702141180&rnid=2619533011&ref=sr_nr_n_11'], 
                 '5_cat_scratcher':['https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975241011%2Cn%3A2975243011%2Cn%3A3024130011&dc&ds=v1%3AKcvLLeg63a6nSZquNC63shIsy%2BGghd3kv8QobyuCcfU&qid=1702141180&rnid=2619533011&ref=sr_nr_n_7',
                                   'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975241011%2Cn%3A2975243011%2Cn%3A2975247011&dc&ds=v1%3A%2B4P%2BaSc5I5EB7aeO1uevvPW82lm%2B54SQadsnf72%2BVuY&qid=1702141180&rnid=2619533011&ref=sr_nr_n_8'], 
                 '6_cat_toilet' :['https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975241011%2Cn%3A2975296011%2Cn%3A2975299011&dc&ds=v1%3ANdsl5u26TJ82%2F7JLNViQgHajgJVOke6VZgQ0%2F%2BbUJ5M&qid=1702141254&rnid=2619533011&ref=sr_nr_n_5'], 
                 '7_house':['https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975312011%2Cn%3A2975400011%2Cn%3A2975402011&dc&ds=v1%3A6mqtLMEB9YX5qDIe04QS5Xke863MTvCDZsK9k%2BrAvGg&qid=1702166892&rnid=2619533011&ref=sr_nr_n_4',
                           'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975241011%2Cn%3A2975243011%2Cn%3A2975246011&dc&ds=v1%3A7oTIrnXa3d8s1hqzx33T33uIBCSzDgb1bvmhFif5XxY&qid=1702171332&rnid=2619533011&ref=sr_nr_n_5'], 
                 '8_carrier':['https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975312011%2Cn%3A2975333011%2Cn%3A2975337011&dc&ds=v1%3ANfod22SDQDgVwK2jGySM1%2Ftxc5rU8pGWtyXdQp9a6Mk&qid=1702168607&rnid=2619533011&ref=sr_nr_n_4', 
                              'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975312011%2Cn%3A2975333011%2Cn%3A2975338011&dc&ds=v1%3A5sOVVgCfylu9iRiruAgP5%2FJwtivoXb%2FzEXByyb0Vs48&qid=1702168607&rnid=2619533011&ref=sr_nr_n_7', 
                             'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975241011%2Cn%3A2975250011%2Cn%3A3024131011&dc&ds=v1%3A1tjmaHMGGEZ3u62A2TpV4%2FSUYqr1xuFGUEJM5t3qYn8&qid=1702171359&rnid=2619533011&ref=sr_nr_n_2',
                             ], 
                 '9_clothes':['https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975312011%2Cn%3A2975313011%2Cn%3A3024173011&dc&ds=v1%3AXfglOl9ynIBEhk%2B%2FAtE9Wiu%2BrG3rFU7cl7QBiuB4sZE&qid=1702170333&rnid=2619533011&ref=sr_nr_n_11',
                             'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975312011%2Cn%3A2975313011%2Cn%3A3024171011&dc&page=5&qid=1702171263&rnid=2619533011&ref=sr_pg_5'
                             'https://www.amazon.com/s?i=pets-intl-ship&bbn=16225013011&rh=n%3A16225013011%2Cn%3A2619533011%2Cn%3A2975241011%2Cn%3A2975242011&dc&ds=v1%3A%2Fe4n0XDlKZL0VgdzQJ59fPU70wIk8kKZ74GfYaB1f8k&qid=1702171307&rnid=2619533011&ref=sr_nr_n_2']}

cate_url_prod_dict = {'2_dog_fence':[], '3_dog_nosework':[], '4_cat_tower':[], 
                 '5_cat_scratcher':[], '6_cat_toilet' :[], '7_house':[], 
                 '8_carrier':[], '9_clothes':[]}

# Selenium을 이용한 Amazon Crawling

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
from urllib.request import urlretrieve, urlopen
import pandas as pd
import os
import numpy as np

# 추천 상품 정보 크롤링
def amazon_prod_url(url, category, max_page):
    browser = webdriver.Chrome()
    browser.get(url)
    sleep(4)
    browser.refresh()
    sleep(4)
    
    hrefs = list() # 상품 페이지 url
    itemname_list = list()
    category_list = list()
    price_list = list()
    rating_list = list()
    reviews_list = list()
    asin_list = list()
    imagefile_list = list() # img hash list
    
    # 상품 진열 페이지에서 각 상품의 url 수집 
    cnt = 0
    while cnt < max_page: # max_page페이지까지만 상품 추출
        
        # 상품 정보 추출
        for i in browser.find_elements(By.CSS_SELECTOR, 'div[data-component-type="s-search-result"]'):
            
            # 상품 url
            href = i.find_element(By.CSS_SELECTOR, 'span[data-component-type="s-product-image"] > a').get_attribute('href')
            hrefs.append(href)
            
            # 상품명
            itemname = i.find_element(By.CSS_SELECTOR, 'span[class="a-size-base-plus a-color-base a-text-normal"]').text
            itemname_list.append(itemname)
            
            # 카테고리
            category_list.append(category)
            
            # 이미지 url
            img = i.find_element(By.CSS_SELECTOR, 'img').get_attribute('srcset').split(', ')
            if img != ['']:
                img = img[1][:-5]
            else:
                img = i.find_element(By.CSS_SELECTOR, 'img').get_attribute('src')
            
            # Asin : 한 번 넣어봄
            asin = i.get_attribute('data-asin')
            asin_list.append(asin)
            
            # 상품 가격
            try:
                price = i.find_element(By.CSS_SELECTOR, 'span[class="a-price"]').text.replace('\n', '.')
            except:
                price = 'None'
                
            price_list.append(price)

            # Rating/Reviews
            try:
                meta_tag = i.find_elements(By.CSS_SELECTOR, 'div[class="a-row a-size-small"] > span')

                if meta_tag != []:
                    rating = meta_tag[0].get_attribute('aria-label').split(' ')[0]
                    reviews = meta_tag[1].get_attribute('aria-label')
                else:
                    rating, reviews = 'None', 'None'
            except: 
                rating, reviews = 'None', 'None'
            
            rating_list.append(rating)
            reviews_list.append(reviews)

            # 이미지 저장
            img_hash = img.split('/')[-1] # url 생성 시: https://m.media-amazon.com/images/W/MEDIAX_792452-T2/images/I/ 
            imagefile_list.append(img_hash)
            
            fdir = f'recommend/{category}/'
            if not os.path.exists(fdir):
                # 디렉토리가 존재하지 않으면 새로 생성
                os.makedirs(fdir)

            file_path = os.path.join(fdir, img_hash)
            urlretrieve(img, file_path)
        
        if browser.find_elements(By.CSS_SELECTOR, 'span[class="s-pagination-strip"] > span')[-1].text == 'Next':
            break
        
        try:
            next_page = browser.find_elements(By.CSS_SELECTOR, 'span[class="s-pagination-strip"] > a')[-1]
            next_page.click()
            sleep(2)
            cnt += 1
        except:
            break
    
    product_DB = pd.DataFrame({'itemname':itemname_list, 'category': category_list, 'price' : price_list, 
                               'rating' : rating_list, 'reviews': reviews_list, 'asin': asin_list, 'imagefile' : imagefile_list})
    
    browser.close()
    
    return hrefs, product_DB

In [9]:
import pickle

cate_url_prod_dict = {'2_dog_fence':[], '3_dog_nosework':[], '4_cat_tower':[], '5_cat_scratcher':[], 
                      '6_cat_toilet' :[], '7_house':[], '8_carrier':[], '9_clothes':[]}

for category, url_list in cate_url_dict.items():
    
    result_csv = pd.DataFrame()
    for url in url_list:
        # 적당한 데이터 수집하기
        if len(url_list) == 1:
            urls , product_DB = amazon_prod_url(url, category, 30)
            
        elif len(url_list) == 2:
            urls , product_DB = amazon_prod_url(url, category,15)
            
        else:
            urls , product_DB = amazon_prod_url(url, category, 10)
            
        cate_url_prod_dict[category].extend(urls)
        result_csv = pd.concat([result_csv, product_DB])
        
    result_csv.to_csv(f'recommend/{category}/result.csv', index = False)
    
    sleep(4)

with open('recommend/cate_url_prod_dict.pickle', 'wb') as fp:
    pickle.dump(cate_url_prod_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
import re
from requests import get, post
from bs4 import BeautifulSoup

def img_crawling(category, url_list):
    fdir = f'similarity/{category}/'
    for url in url_list:
        if category == '2_dog_fence':
            break
            
        if len(os.listdir(fdir)) > 4000:
            break
            
        resp = get(url)
        dom = BeautifulSoup(resp.text, 'html5lib')
        
        for img in dom.select('img'):
            try:
                img = img.attrs['src']
                img_hash = img.split('/')[-1]
                if re.search('.jpg', img):
                    print(f'{category} : {img_hash}')
                    if not os.path.exists(fdir):
                    # 디렉토리가 존재하지 않으면 새로 생성
                        os.makedirs(fdir)

                    file_path = os.path.join(fdir, img_hash)
                    urlretrieve(img, file_path)
            except:
                pass

In [88]:
for category, url_list in cate_url_prod_dict.items():
    img_crawling(category, url_list)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SamSung\AppData\Local\Temp\ipykernel_14024\697211400.py", line 2, in <module>
    img_crawling(category, url_list)
  File "C:\Users\SamSung\AppData\Local\Temp\ipykernel_14024\656797559.py", line 8, in img_crawling
    dom = BeautifulSoup(resp.text, 'html5lib')
                        ^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\requests\models.py", line 928, in text
    encoding = self.apparent_encoding
               ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\requests\models.py", line 793, in apparent_encoding
    return chardet.detect(self.content)["encoding"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\chardet\__init__.py", line 41, in detect
    detector.feed(byte_str)
  File "C

In [2]:
import pickle

with open('recommend/cate_url_prod_dict.pickle', 'rb') as fp:
    cate_url_prod_dict = pickle.load(fp)

In [3]:
cate_url_prod_dict

{'2_dog_fence': ['https://www.amazon.com/sspa/click?ie=UTF8&spc=MTozODUzNjY0MTMwMzk4NzA4OjE3MDIyNDgxNjY6c3BfYXRmX2Jyb3dzZToyMDAxNjQ1MDgxOTA2OTg6OjA6Og&url=%2FPUPETPO-Freestanding-Foldable-Doorways-Included%2Fdp%2FB0BBL2V9RM%2Fref%3Dsr_1_1_sspa%3Fqid%3D1702248166%26rnid%3D2619533011%26s%3Dpet-supplies%26sr%3D1-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1',
  'https://www.amazon.com/sspa/click?ie=UTF8&spc=MTozODUzNjY0MTMwMzk4NzA4OjE3MDIyNDgxNjY6c3BfYXRmX2Jyb3dzZTozMDAxMDA2MTA0ODk4MDI6OjA6Og&url=%2FMino-Kesper-Wire-Dog-Gate%2Fdp%2FB0CL4C57KY%2Fref%3Dsr_1_2_sspa%3Fqid%3D1702248166%26rnid%3D2619533011%26s%3Dpet-supplies%26sr%3D1-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1',
  'https://www.amazon.com/sspa/click?ie=UTF8&spc=MTozODUzNjY0MTMwMzk4NzA4OjE3MDIyNDgxNjY6c3BfYXRmX2Jyb3dzZToyMDAxMjgyMjY4NDA4OTg6OjA6Og&url=%2FPUPETPO-Freestanding-Support-Triangle-Foldable%2Fdp%2FB0BQ6417W5%2Fref%3Dsr_1_3_sspa%3Fqid%3D1702248166%26rnid%3D2619533011%26s%3Dpet-supplies%2

In [21]:
# 수집한 상품 url 개수
for k, v in cate_url_prod_dict.items():
    print(len(v))

990
390
990
990
990
911
914
785


In [345]:
# 실제 상품 이미지 개수 : 중복 상당수 존재
cate_list = ['2_dog_fence', '3_dog_nosework', '4_cat_tower', '5_cat_scratcher', 
             '6_cat_toilet' , '7_house', '8_carrier', '9_clothes']
for cate in cate_list:
    print(len(os.listdir(f'recommend/{cate}')))

310
31
303
323
301
311
320
325


In [3]:
cate_url_prod_dict['5_cat_scratcher'].index('https://www.amazon.com/nazva-Sisal-Scratch-Pad-Beige/dp/B081V1ZMHJ/ref=sr_1_221?qid=1702250552&rnid=2619533011&s=pet-supplies&sr=1-221')

292

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
from urllib.request import urlretrieve, urlopen

def amazon_prod_img_down(category, url):
    try:
        browser = webdriver.Chrome()
        browser.get(url)
        sleep(2)
        browser.refresh()
        sleep(2)

        try:
            small = browser.find_elements(By.CSS_SELECTOR, 'li[class="a-spacing-small item imageThumbnail a-declarative"]')
            for sm in small:
                sm.click()

            meta = browser.find_element(By.CSS_SELECTOR, 'ul[class="a-unordered-list a-nostyle a-horizontal list maintain-height"]')
            imgs = meta.find_elements(By.CSS_SELECTOR, 'img')
            try:
                for img in imgs:
                    try:
                        img = img.get_attribute('src')
                        img_hash = img.split('/')[-1]

                        fdir = f'similarity/{category}/'
                        if not os.path.exists(fdir):
                            # 디렉토리가 존재하지 않으면 새로 생성
                            os.makedirs(fdir)

                        file_path = os.path.join(fdir, img_hash)
                        urlretrieve(img, file_path)

                    except:
                        print(f'{url} : 이미지 다운로드 실패')
                        continue
                        
                print(f'{url} : 이미지 다운로드 성공')
                browser.quit()
                
            except:
                
                print(f'{url} : 이미지 다운로드 실패')
                browser.quit()

        except:

            print(f'{url} : 이미지 다운로드 실패')
            browser.quit()
    
    except:
        
        pass
    
    return

In [ ]:
i = 0
for url in cate_url_prod_dict['9_clothes'][:100]:      
    amazon_prod_img_down('9_clothes', url)
    print(i)
    i += 1

# for category, url_list in cate_url_prod_dict.items():
    
#     if category in ['2_dog_fence', '3_dog_nosework', '4_cat_tower', '5_cat_scratcher', '6_cat_toilet', '7_house']:
#         continue
        
#     i = 0
#     for url in url_list: # 6_cat_toilt 수집
#         amazon_prod_img_down(category, url)
#         print(i)
#         i += 1

https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo1NDQzMzMzNDY1ODYxNzA5OjE3MDIyNTMxNDY6c3BfYXRmX2Jyb3dzZToyMDAxNTU0Mjc1NjUxOTg6OjA6Og&url=%2FDog-Life-Jacket-Reflective-Adjustable%2Fdp%2FB0BYNPJTGG%2Fref%3Dsr_1_1_sspa%3Fqid%3D1702253146%26rnid%3D2619533011%26s%3Dpet-supplies%26sr%3D1-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1 : 이미지 다운로드 성공
0
https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo1NDQzMzMzNDY1ODYxNzA5OjE3MDIyNTMxNDY6c3BfYXRmX2Jyb3dzZTozMDAwNzI0MDE5NTU4MDI6OjA6Og&url=%2FGEEZO-Protection-Cancelling-Headphones-Fireworks%2Fdp%2FB0CF4Q1H6Z%2Fref%3Dsr_1_2_sspa%3Fqid%3D1702253146%26rnid%3D2619533011%26s%3Dpet-supplies%26sr%3D1-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1 : 이미지 다운로드 성공
1
https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo1NDQzMzMzNDY1ODYxNzA5OjE3MDIyNTMxNDY6c3BfYXRmX2Jyb3dzZTozMDAwMjA5MjMyMDMxMDI6OjA6Og&url=%2FOwnpets-Inflatable-Lightweight-Adjustable-Reflective%2Fdp%2FB0C2V6G328%2Fref%3Dsr_1_3_sspa%3Fqid%3D1702253146%26rnid%3D26195330

In [11]:
cate_url_prod_dict['5_cat_scratcher'][149]

'https://www.amazon.com/sspa/click?ie=UTF8&spc=MToyMjI0Mjg3NzI0MzExNzA4OjE3MDIyNTA0MzU6c3BfbXRmX2Jyb3dzZToyMDAwODU4MTYwMjkwNjE6OjA6Og&url=%2FScratcher-Cardboard-Reversible-Recyclable-Suitable%2Fdp%2FB09BYZPG96%2Fref%3Dsr_1_114_sspa%3Fqid%3D1702250435%26rnid%3D2619533011%26s%3Dpet-supplies%26sr%3D1-114-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9tdGZfYnJvd3Nl%26psc%3D1'

In [ ]:
for category, url_list in cate_url_prod_dict.items():
    
    if category in ['2_dog_fence', '3_dog_nosework', '4_cat_tower']:
        continue
        
    for url in url_list:
        amazon_prod_img_down(category, url)

In [ ]:
https://www.amazon.com/KZLAA-Furniture-Scratch-Natural-Climbing/dp/B0C32PV4JY/ref=sr_1_169_sspa?qid=1702249566&rnid=2619533011&s=pet-supplies&sr=1-169-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl&psc=1

In [15]:
url = 'https://www.amazon.com/sspa/click?ie=UTF8&spc=MTozODUzNjY0MTMwMzk4NzA4OjE3MDIyNDgxNjY6c3BfYXRmX2Jyb3dzZTozMDAxMDA2MTA0ODk4MDI6OjA6Og&url=%2FMino-Kesper-Wire-Dog-Gate%2Fdp%2FB0CL4C57KY%2Fref%3Dsr_1_2_sspa%3Fqid%3D1702248166%26rnid%3D2619533011%26s%3Dpet-supplies%26sr%3D1-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGZfYnJvd3Nl%26psc%3D1'
browser = webdriver.Chrome()
browser.get(url)
sleep(3)
browser.refresh()
sleep(3)

In [62]:
small = browser.find_elements(By.CSS_SELECTOR, 'li[class="a-spacing-small item imageThumbnail a-declarative"]')
for sm in small:
    sm.click()

meta = browser.find_element(By.CSS_SELECTOR, 'ul[class="a-unordered-list a-nostyle a-horizontal list maintain-height"]')
imgs = meta.find_elements(By.CSS_SELECTOR, 'img')

In [63]:
for img in imgs:
    print(img.get_attribute('src'))

https://m.media-amazon.com/images/W/MEDIAX_792452-T1/images/I/819KWLzC9bL._AC_SX425_.jpg
https://m.media-amazon.com/images/W/MEDIAX_792452-T1/images/I/81jWAd1HohL._AC_SX425_.jpg
https://m.media-amazon.com/images/W/MEDIAX_792452-T1/images/I/81LfUt5T+XL._AC_SX425_.jpg
https://m.media-amazon.com/images/W/MEDIAX_792452-T1/images/I/71nbfd5gPWL._AC_SX425_.jpg
https://m.media-amazon.com/images/W/MEDIAX_792452-T1/images/I/71modFwIX7L._AC_SX425_.jpg
https://m.media-amazon.com/images/W/MEDIAX_792452-T1/images/I/71+-ZAOsCmL._AC_SX425_.jpg


In [23]:
meta

[<selenium.webdriver.remote.webelement.WebElement (session="7d94bb02ab735089041c0d00f729a2d1", element="8CD98B3B12162084F4756009F803E299_element_2282")>]

In [24]:
browser.find_element(By.CSS_SELECTOR, 'li[data-csa-c-action="image-block-main-image-hover"]')

<selenium.webdriver.remote.webelement.WebElement (session="7d94bb02ab735089041c0d00f729a2d1", element="8CD98B3B12162084F4756009F803E299_element_463")>

In [27]:
browser.find_element(By.CSS_SELECTOR, 'li[data-csa-c-action="image-block-main-image-hover"]')

[<selenium.webdriver.remote.webelement.WebElement (session="7d94bb02ab735089041c0d00f729a2d1", element="2E0E656DA128D4986D782233FD7DD90F_element_6706")>]

In [59]:
small = browser.find_elements(By.CSS_SELECTOR, 'li[class="a-spacing-small item imageThumbnail a-declarative"]')
for sm in small:
    sm.click()

In [32]:
meta

[<selenium.webdriver.remote.webelement.WebElement (session="7d94bb02ab735089041c0d00f729a2d1", element="8CD98B3B12162084F4756009F803E299_element_2282")>]